# Résolution Pb57 de la collection Hock-Schittkowski

Classification dans l'ouvrage : SQR-P1-1

Number of variables : n = 2

Number of constraints : 3 (1 inequality + 2 bounds)

# Modèle

$\left\{  
\begin{array}{llll} 
\min f(x) \\ 
\text{s.c.}\\
0.49x_2-x_1x_2-0.09 &\geq 0\\
x_1\geq 0.4,\  x_2 \geq -4
\end{array} \right.$

où $f : x \longmapsto \sum\limits_{i=1}^{44} f_i(x)^2$

avec $f_i(x) = b_i - x_1 - (0.49-x_1)\exp(-x_2(a_i-8)) \text{ pour }i=1,\ldots,44$ 

Point de départ : $x_0 = (0.42,5)$

Solution attendue : $x^* = (0.419952675,1.284845629)$

Fonction objectif à la solution : $f(x^*) =0.02845966972$

# Résolution avec ENLSIP-0.2.0

In [5]:
using JuMP, Ipopt, LinearAlgebra, Polynomials, Printf, Plots, BenchmarkTools

In [15]:
path_enlsip_file = string(pwd(),"/enlsip-julia-0.2.0.jl")
include(path_enlsip_file)

In [7]:
n = 2
m = 44
nb_eq = 0 # nombre de contraintes d'égalité
nb_constraints = 3 # nombre de contraintes d'égalité et d'inégalité

3

### Données d'entrée

In [8]:
a = [8.,8.,10.,10.,10.,10.,12.,12.,12.,12.,14.,14.,14.,16.,16.,16.,18.,18.,20.,20.,20.,22.,22.,22., 
         24.,24.,24.,26.,26.,26.,28.,28.,30.,30.,30.,32.,32.,34.,36.,36.,38.,38.,40.,42.]

b = [.49,.49,.48,.47,.48,.47,.46,.46,.45,.43,.45,.43,.43,.44,.43,.43,.46,.45,.42,.42,.43,.41,
         .41,.40,.42,.40,.40,.41,.40,.41,.41,.40,.40,.40,.38,.41,.40,.40,.41,.38,.40,.40,.39,.39]

44-element Vector{Float64}:
 0.49
 0.49
 0.48
 0.47
 0.48
 0.47
 0.46
 0.46
 0.45
 0.43
 0.45
 0.43
 0.43
 ⋮
 0.4
 0.4
 0.38
 0.41
 0.4
 0.4
 0.41
 0.38
 0.4
 0.4
 0.39
 0.39

In [9]:
# Résidus et matrice jacobienne associée

r_i(x::Vector,t::Float64) = x[1] + (0.49 - x[1]) * exp(-x[2]*(t - 8))
res57 = ResidualsEval(0)

function (res57::ResidualsEval)(x::Vector,rx::Vector,J::Matrix)
    # Evaluate the residuals
    if abs(res57.ctrl) == 1
        rx[:] = b - (t::Float64 -> r_i(x,t)).(a)

    # The jacobian is computed numericaly using forward differences
    # ctrl is set to 0
    elseif res57.ctrl == 2 res57.ctrl = 0 end
    return
end

In [10]:
# Contraintes et matrice jacobienne associée

cons57 = ConstraintsEval(0)

function (cons57::ConstraintsEval)(x::Vector,cx::Vector,A::Matrix)
    # Evaluate the constraints
    if abs(cons57.ctrl) == 1
        cx[:] = [0.49 * x[2] - x[1] * x[2] - 0.09, x[1] - 0.4, x[2] + 4]
    
    # The jacobian is computed anaticaly
    elseif cons57.ctrl == 2
        A[:] = [-x[2] 0.49-x[1];
        1.0 0.0;
        0.0 1.0]
    end
end

# Résolution avec ENLSIP-0.2.0

In [16]:
x0 = [0.42, 5.0]
enlsip_020(x0,res57,cons57,n,m,nb_eq,nb_constraints)

****************************************
*                                      *
*          ENLSIP-JULIA-0.2.0          *
*                                      *
****************************************

Starting point : [0.42, 5.0]

Number of equality constraints   : 0
Number of inequality constraints : 3

iter    objective    cx_sum   method   ||p||   dimA  dimJ2     α     conv. speed   max weight   working set
   0  2.845938e-02  0.00e+00     1   9.233e+03   0     2   4.02e-04    0.00e+00     1.00e-01     -
   1  2.845958e-02  3.03e-11     1   7.321e-04   1     1   7.56e-01    2.24e-03     2.16e+04    (1)
   2  2.845967e-02  1.86e-12     1   2.021e-04   1     1   1.00e+00    2.76e-01     9.48e+04    (1)
   3  2.845967e-02  5.91e-18     1   8.677e-06   1     1   1.00e+00    4.29e-02     5.34e+07    (1)
   4  2.845967e-02  1.69e-23     1   3.804e-07   1     1   1.00e+00    4.38e-02     3.15e+10    (1)

Exit code = 40
Number of iterations = 5 

Terminated at point : 4.199527e-01  1.2

In [9]:
x1_sol = enlsip_020.sol[1]
x2_sol = enlsip_020.sol[2]

println("Solution obtenue avec ENLSIP :")
println("x_sol = $(enlsip_020.sol)")
println("Valeur de l'objecif : $(enlsip_020.obj_value)","\n")
println("Evaluation des contraintes : $([0.49 * x2_sol - x1_sol * x2_sol - 0.09, x1_sol - 0.4, x2_sol + 4])")

Solution obtenue avec ENLSIP :
x_sol = [0.4199526506972085, 1.2848451925134212]
Valeur de l'objecif : 0.028459669722986667

Evaluation des contraintes : [8.326672684688674e-17, 0.019952650697208474, 5.284845192513421]


In [10]:
println("Solution trouvée avec ENLSIP :")
@printf "x_sol = (%.8e, %.9e)\n" x1_sol x2_sol
@printf "f(x_sol) = %.9e\n" enlsip_020.obj_value  
println("\nValeurs théoriques visées :")
@printf "x_opt = (%.8e, %.9e)\n" 0.419952675 1.284845629
@printf "f(x_opt) = %.9e" 0.02845966972

Solution trouvée avec ENLSIP :
x_sol = (4.19952651e-01, 1.284845193e+00)
f(x_sol) = 2.845966972e-02

Valeurs théoriques visées :
x_opt = (4.19952675e-01, 1.284845629e+00)
f(x_opt) = 2.845966972e-02

# Résolution avec Ipopt

In [11]:
model = Model(with_optimizer(Ipopt.Optimizer))
@variable(model, x1, start = 0.42)
@variable(model, x2, start = 5.)

function f_i(x1, x2, t::Float64)
    return x1 + (0.49 - x1) * exp(-x2*(t - 8))
end

function f(x1, x2)
    y = b - (t::Float64 -> f_i(x1,x2,t)).(a)
    return dot(y,y) 
end

JuMP.register(model, :f, 2, f, autodiff=true)

@NLconstraint(model, c1, 0.49*x2 - x1*x2 - 0.09 >= 0)
@constraint(model, x1 >= 0.4)
@constraint(model, x2 >= -4)

@NLobjective(model, Min, f(x1,x2))

JuMP.optimize!(model)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        4
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equal

In [12]:
println("Solution ENLSIP-Julia :")
@printf "x_sol = (%.8e, %.9e)\n" x1_sol x2_sol
@printf "f(x_sol) = %.9e\n" enlsip_020.obj_value 
println("\nSolution Ipopt :")
@printf "x_opt = (%.8e, %.9e)\n" JuMP.value(x1) JuMP.value(x2)
@printf "f(x_opt) = %.9e " 0.028459669065836156

Solution ENLSIP-Julia :
x_sol = (4.19952651e-01, 1.284845193e+00)
f(x_sol) = 2.845966972e-02

Solution Ipopt :
x_opt = (4.19952650e-01, 1.284845043e+00)
f(x_opt) = 2.845966907e-02 